In [1]:
# TUTORIAL: https://www.youtube.com/watch?v=Xjv1sY630Uc
# https://github.com/CarsonLloydd/Web-Scraping-Zillow/blob/master/WS_Zillow_KC_houses_CL.ipynb\
#https://github.com/CarsonLloydd/Web-Scraping-Zillow/blob/master/WS_Zillow_KC_houses_CL.ipynb
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import requests
from bs4 import BeautifulSoup
import json
import time
import csv


In [6]:
#"https://www.zillow.com/king-county-wa/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22mapBounds%22%3A%7B%22west%22%3A-122.69592584911955%2C%22east%22%3A-120.89801315088046%2C%22south%22%3A46.85500011855959%2C%22north%22%3A48.005794108822954%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A207%2C%22regionType%22%3A4%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A9%7D"
#url = 'https://www.zillow.com/king-county-wa/?searchQueryState={%22pagination%22:{},%22usersSearchTerm%22:%22King%20County,%20WA%22,%22mapBounds%22:{%22west%22:-122.67175587695311,%22east%22:-120.92218312304686,%22south%22:46.89011178033753,%22north%22:47.971419225684436},%22regionSelection%22:[{%22regionId%22:207,%22regionType%22:4}],%22mapZoom%22:9,%22filterState%22:{%22sortSelection%22:{%22value%22:%22globalrelevanceex%22}},%22isListVisible%22:true,%22isMapVisible%22:true}'
url = 'https://www.zillow.com/saint-petersburg-fl/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Saint%20Petersburg%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.95780176685055%2C%22east%22%3A-82.41809839770993%2C%22south%22%3A27.60909405529139%2C%22north%22%3A27.985322250107767%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A26922%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D'
res = requests.get(url)


In [7]:
print(res.raise_for_status())
soup = BeautifulSoup(res.text)
type(soup)

None


bs4.BeautifulSoup

In [8]:
print(soup.prettify())


<html>
 <head>
  <meta content="noindex, nofollow" name="robots"/>
  <link href="https://www.zillowstatic.com/vstatic/80d5e73/static/css/z-pages/captcha.css" media="screen" rel="stylesheet" type="text/css"/>
  <script>
   window._pxAppId = 'PXHYx10rg3';
        window._pxJsClientSrc = '/HYx10rg3/init.js';
        window._pxHostUrl = '/HYx10rg3/xhr';
        window._pxFirstPartyEnabled = true;
        window._pxreCaptchaTheme='light';
  </script>
  <script src="https://captcha.px-cdn.net/PXHYx10rg3/captcha.js?a=c&amp;m=0" type="text/javascript">
  </script>
  <script>
   function getQueryString(name, url) {
            if (!url) url = window.location.href;
            name = name.replace(/[\[\]]/g, "\\$&");
            var regex = new RegExp("[?&]" + name + "(=([^&#]*)|&|#|$)"),
                results = regex.exec(url);
            if (!results) return null;
            if (!results[2]) return '';
            return decodeURIComponent(results[2].replace(/\+/g, " "));
        }
        

In [10]:
!pip install selenium

In [18]:
import selenium

In [61]:
from selenium import webdriver
import sys
from selenium.webdriver.chrome.options import Options
import re

In [24]:
chrome_path = "C:\Program Files (x86)\chromedriver.exe"

In [25]:
#driver = webdriver.Chrome(PATH)
driver = webdriver.Chrome(chrome_path, options=options)

In [26]:
driver.get(url)

In [27]:
driver.find_element_by_class_name('list-card-heading')

<selenium.webdriver.remote.webelement.WebElement (session="49fa92128027f014550a4f489bbf7290", element="23f643e7-9121-4385-894d-1b4c2dea965a")>

In [28]:
page_source = driver.page_source
soup = BeautifulSoup(page_source)

In [29]:
#price
float(soup.findAll('div', {'class' : 'list-card-price'})[0].contents[0].replace('$', '').replace(',', ''))

259900.0

In [30]:
#beds
float(soup.findAll('ul', {'class' : 'list-card-details'})[0].contents[0].contents[0])

3.0

In [31]:
#baths
float(soup.findAll('ul', {'class' : 'list-card-details'})[0].contents[1].contents[0])

2.0

In [32]:
#sqft
float(soup.findAll('ul', {'class' : 'list-card-details'})[0].contents[2].contents[0].replace(',', ''))

1398.0

In [89]:
# there are 40 listings per page, thats why the range is 40
# some listings may have missing information, that's why there are exceptions
x = 0
bedrooms = []
bathrooms = []
sqft = []
price = []
p = list(range(40))
for v in p:
    if len(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents) == 3:
        try:
            price.append(float(soup.findAll('div', {'class' : 'list-card-price'})[x].contents[0].replace('$', '').replace(',', '')))
            bedrooms.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[0].contents[0]))
            bathrooms.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[1].contents[0]))
            sqft.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[2].contents[0].replace(',', '')))
            x = x + 1
        except:
            if len(price) > len(bedrooms):
                bedrooms.append(None)
                bathrooms.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[1].contents[0]))
                sqft.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[2].contents[0].replace(',', '')))
                x = x + 1
            elif len(bedrooms) > len(bathrooms):
                bathrooms.append(None)
                sqft.append(float(soup.findAll('ul', {'class' : 'list-card-details'})[x].contents[2].contents[0].replace(',', '')))
                x = x + 1
            else:
                sqft.append(None)
                x = x + 1
            continue
    else:
        x = x + 1

In [101]:
for i in soup.find_all('script', type='application/ld+json'):
    print([i])

[<script type="application/ld+json">{"@type":"SingleFamilyResidence","@context":"http://schema.org","name":"600 17th Ave S, Saint Petersburg, FL 33701","floorSize":{"@type":"QuantitativeValue","@context":"http://schema.org","value":"1,398"},"address":{"@type":"PostalAddress","@context":"http://schema.org","streetAddress":"600 17th Ave S","addressLocality":"Saint Petersburg","addressRegion":"FL","postalCode":"33701"},"geo":{"@type":"GeoCoordinates","@context":"http://schema.org","latitude":27.75326,"longitude":-82.640567},"url":"https://www.zillow.com/homedetails/600-17th-Ave-S-Saint-Petersburg-FL-33701/47230422_zpid/"}</script>]
[<script type="application/ld+json">{"@type":"Event","@context":"http://schema.org","name":"3D Tour Available - 600 17th Ave S","url":"https://www.zillow.com/homedetails/600-17th-Ave-S-Saint-Petersburg-FL-33701/47230422_zpid/","image":"https://photos.zillowstatic.com/fp/0a3416355e863317a76437cc5dc131ec-p_e.jpg","startDate":"2021-01-29","endDate":"2021-01-29","e

In [118]:
import json

In [121]:
houses2 = soup.findAll('script', {'type' : 'application/ld+json'})


In [122]:
houses2

[<script type="application/ld+json">{"@type":"SingleFamilyResidence","@context":"http://schema.org","name":"600 17th Ave S, Saint Petersburg, FL 33701","floorSize":{"@type":"QuantitativeValue","@context":"http://schema.org","value":"1,398"},"address":{"@type":"PostalAddress","@context":"http://schema.org","streetAddress":"600 17th Ave S","addressLocality":"Saint Petersburg","addressRegion":"FL","postalCode":"33701"},"geo":{"@type":"GeoCoordinates","@context":"http://schema.org","latitude":27.75326,"longitude":-82.640567},"url":"https://www.zillow.com/homedetails/600-17th-Ave-S-Saint-Petersburg-FL-33701/47230422_zpid/"}</script>,
 <script type="application/ld+json">{"@type":"Event","@context":"http://schema.org","name":"3D Tour Available - 600 17th Ave S","url":"https://www.zillow.com/homedetails/600-17th-Ave-S-Saint-Petersburg-FL-33701/47230422_zpid/","image":"https://photos.zillowstatic.com/fp/0a3416355e863317a76437cc5dc131ec-p_e.jpg","startDate":"2021-01-29","endDate":"2021-01-29","e

In [123]:
import ast

_dict = ast.literal_eval(houses2)
_dict

ValueError: malformed node or string: [<script type="application/ld+json">{"@type":"SingleFamilyResidence","@context":"http://schema.org","name":"600 17th Ave S, Saint Petersburg, FL 33701","floorSize":{"@type":"QuantitativeValue","@context":"http://schema.org","value":"1,398"},"address":{"@type":"PostalAddress","@context":"http://schema.org","streetAddress":"600 17th Ave S","addressLocality":"Saint Petersburg","addressRegion":"FL","postalCode":"33701"},"geo":{"@type":"GeoCoordinates","@context":"http://schema.org","latitude":27.75326,"longitude":-82.640567},"url":"https://www.zillow.com/homedetails/600-17th-Ave-S-Saint-Petersburg-FL-33701/47230422_zpid/"}</script>, <script type="application/ld+json">{"@type":"Event","@context":"http://schema.org","name":"3D Tour Available - 600 17th Ave S","url":"https://www.zillow.com/homedetails/600-17th-Ave-S-Saint-Petersburg-FL-33701/47230422_zpid/","image":"https://photos.zillowstatic.com/fp/0a3416355e863317a76437cc5dc131ec-p_e.jpg","startDate":"

In [75]:
# each house listing has a script that contained extra 
# information like the url, zipcode, latitude and longitude
houses = soup.findAll('script', {'type' : 'application/ld+json'})

print(houses[0])

<script type="application/ld+json">{"@type":"SingleFamilyResidence","@context":"http://schema.org","name":"600 17th Ave S, Saint Petersburg, FL 33701","floorSize":{"@type":"QuantitativeValue","@context":"http://schema.org","value":"1,398"},"address":{"@type":"PostalAddress","@context":"http://schema.org","streetAddress":"600 17th Ave S","addressLocality":"Saint Petersburg","addressRegion":"FL","postalCode":"33701"},"geo":{"@type":"GeoCoordinates","@context":"http://schema.org","latitude":27.75326,"longitude":-82.640567},"url":"https://www.zillow.com/homedetails/600-17th-Ave-S-Saint-Petersburg-FL-33701/47230422_zpid/"}</script>


In [97]:
        print(tmp)

None


In [76]:
type(houses)

bs4.element.ResultSet

In [105]:

soup2 = BeautifulSoup(houses)
print(soup2.prettify())

TypeError: 'NoneType' object is not callable